In [317]:
#
# Verifica se a media horária de ônibus em circulação por linha atingiu a meta horária 
# estabelecida
#
# Abaixo as informações geradas:
# 1. Lista de linhas que atingiram a meta estabelecida
# 2. Lista de linhas que estão abaixo da meta estabelecida
# 3. Total de linhas que atingiram a meta estabelecida
# 4. Total de linhas que estão abaixo da meta estabelecida
#
# *Obs: a meta estabelecida encontra-se na tabela META_ONIBUS_POR_LINHA
#


In [318]:
from pyspark.sql import SparkSession

In [319]:
from pyspark.sql.functions import explode

In [320]:
from pyspark.sql.functions import from_utc_timestamp,split, substring, sum, avg, max,filter, ceil, col, count

In [321]:
from datetime import datetime, timedelta
import zoneinfo as zi

In [322]:
spark = SparkSession.builder.appName("FIA-Proj-SPTRANS-Ouro-Meta").enableHiveSupport().getOrCreate()

In [323]:
#####################################################################################################
#
# Calcula a data/hora para verificação do atingimento da meta. 
# Esse cálculo é baseado em números de horas anteriores a corrente.
#
#parser= argparse.ArgumentParser()

#parser.add_argument("hora", help="qtde de horas anteriores a ser processada", type=int )
#args= parser.parse_args()
#hora_ant= args.hora

#seta o número de horas anteriores a corrente, em que a verificação do atingimento da meta 
#será verificada
hora_ant=16

#Calcula a hora a ser processada
GMT = zi.ZoneInfo('GMT')
LOCAL_TZ_STR='America/Sao_Paulo'
LOCAL_TZ = zi.ZoneInfo(LOCAL_TZ_STR)

dt_localtime=datetime.now(tz=LOCAL_TZ)
dt_lasthour= dt_localtime - timedelta(hours=hora_ant)

str_dt_lasthour= dt_lasthour.strftime('%Y/%m/%d/%H')
str_lasthour= dt_lasthour.strftime('%H')
str_date= dt_lasthour.strftime('%Y%m%d')

In [324]:
#####################################################################################################
#
# Seta o path da camada ouro onde estão as informações da média de ônibus em circulação da hora a
# ser verificada. Também são setados os paths onde serão armazenados as informações abaixo:
#
# 1. Lista de linhas que atingiram a meta estabelecida
# 2. Lista de linhas que estão abaixo da meta estabelecida
# 3. Total de linhas que atingiram a meta estabelecida
# 4. Total de linhas que estão abaixo da meta estabelecida
#


In [325]:
#Seta o path da camada ouro onde foram persistidos as informacões de média de ônibus em circulação
#por linha da hora a ser verificada
ouro_avg_linha= 's3a://ouro/MEDIA_ONIBUS_POR_LINHA/' +  str_dt_lasthour + "/"

In [326]:
#Seta o path da camada ouro onde serão persistidos a lista das linhas que atingiram a meta
#estabelecida
ouro_lista_linhas_ok= 's3a://ouro/LINHAS_ATINGIDA_META/' +  str_dt_lasthour + "/"

In [327]:
#Seta o path da camada ouro onde serão persistidos a lista das linhas que não atingiram
#a meta estabelecida
ouro_lista_linhas_abaixo= 's3a://ouro/LINHAS_ABAIXO_META/' +  str_dt_lasthour + "/"

In [328]:
#Seta o path da camada ouro onde serão persistidos o total de linhas que atingiram  
#a meta estabelecida 
ouro_total_linhas_ok= 's3a://ouro/TOTAL_LINHAS_ATINGIDA_META/' +  str_dt_lasthour + "/"

In [329]:
#Seta o path da camada ouro onde serão persistidos o total de linhas que estão abaixo da meta
#estabelecida 
ouro_total_linhas_abaixo= 's3a://ouro/TOTAL_LINHAS_ABAIXO_META/' +  str_dt_lasthour + "/"

In [330]:
print(ouro_avg_linha)
print(ouro_lista_linhas_ok)
print(ouro_lista_linhas_abaixo)
print(ouro_total_linhas_ok)
print(ouro_total_linhas_abaixo)

s3a://ouro/MEDIA_ONIBUS_POR_LINHA/2024/09/20/22/
s3a://ouro/LINHAS_ATINGIDA_META/2024/09/20/22/
s3a://ouro/LINHAS_ABAIXO_META/2024/09/20/22/
s3a://ouro/TOTAL_LINHAS_ATINGIDA_META/2024/09/20/22/
s3a://ouro/TOTAL_LINHAS_ABAIXO_META/2024/09/20/22/


In [331]:
#####################################################################################################
#
# Configuração das informações de conexão com a base de dados Postgres
#

In [332]:
url = "jdbc:postgresql://db:5432/dvdrental"

properties = {
"user": "admin",
"password": "admin",
"driver": "org.postgresql.Driver"
}

In [333]:
#####################################################################################################
#
# Leitura da camada ouro com as informações estatísticas da quantidade de ônibus em circulação 
# na data/hora calculada
#

In [334]:
df_avg_linha= spark.read.parquet(ouro_avg_linha)

In [335]:
#Exibe o schema 
df_avg_linha.printSchema()

root
 |-- data_ref: string (nullable = true)
 |-- hora_id_ref: string (nullable = true)
 |-- id_linha: long (nullable = true)
 |-- sentido_linha: long (nullable = true)
 |-- let_cod_linha: string (nullable = true)
 |-- let_destino: string (nullable = true)
 |-- let_origem: string (nullable = true)
 |-- qtde_onibus: long (nullable = true)



In [336]:
df_avg_linha.count()

2061

In [337]:
df_avg_linha.show(5)

+--------+-----------+--------+-------------+-------------+--------------------+----------------+-----------+
|data_ref|hora_id_ref|id_linha|sentido_linha|let_cod_linha|         let_destino|      let_origem|qtde_onibus|
+--------+-----------+--------+-------------+-------------+--------------------+----------------+-----------+
|20240920|         22|   32863|            2|      6034-10|        TERM. GRAJAÚ| PQ. RES. COCAIA|          5|
|20240920|         22|    2540|            1|      8029-10|       TERM. MORUMBI|    SHOP. PORTAL|          4|
|20240920|         22|      13|            1|      6836-10|     TERM. JOÃO DIAS|   CAPÃO REDONDO|          5|
|20240920|         22|   33481|            2|      2100-10|          PÇA. DA SÉ|TERM. VL. CARRÃO|          8|
|20240920|         22|     340|            1|      4727-10|METRÔ PÇA. DA ÁRVORE|      JD. CLÍMAX|          2|
+--------+-----------+--------+-------------+-------------+--------------------+----------------+-----------+
only showi

In [338]:
# Seleciona as colunas da hora, código da linha e qtde de ônibus
df_avg_linha_sel=  df_avg_linha.select(
    col("data_ref"),
    col("hora_id_ref").alias('avg_hora_id_ref'),
    col("id_linha").alias('avg_id_linha'),
    col("qtde_onibus").alias('avg_qtde_onibus'))

In [339]:
df_avg_linha_sel.printSchema()

root
 |-- data_ref: string (nullable = true)
 |-- avg_hora_id_ref: string (nullable = true)
 |-- avg_id_linha: long (nullable = true)
 |-- avg_qtde_onibus: long (nullable = true)



In [340]:
df_avg_linha_sel.show(5)

+--------+---------------+------------+---------------+
|data_ref|avg_hora_id_ref|avg_id_linha|avg_qtde_onibus|
+--------+---------------+------------+---------------+
|20240920|             22|       32863|              5|
|20240920|             22|        2540|              4|
|20240920|             22|          13|              5|
|20240920|             22|       33481|              8|
|20240920|             22|         340|              2|
+--------+---------------+------------+---------------+
only showing top 5 rows



In [341]:
#####################################################################################################
#
# Lê a tabela do PostgreSql com a meta horária de ônibus em circulação por linha
#

In [342]:
df_meta_linha= spark.read.jdbc( url=url, table='sptrans.meta_onibus_por_linha',properties= properties)

In [343]:
#Exibe o schema 
df_meta_linha.printSchema()

root
 |-- hora_id_ref: string (nullable = true)
 |-- id_linha: long (nullable = true)
 |-- sentido_linha: long (nullable = true)
 |-- let_cod_linha: string (nullable = true)
 |-- let_destino: string (nullable = true)
 |-- let_origem: string (nullable = true)
 |-- meta_qtde_onibus: long (nullable = true)



In [344]:
df_meta_linha.count()

12651

In [345]:
df_meta_linha.show(5)

+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+
|hora_id_ref|id_linha|sentido_linha|let_cod_linha|        let_destino|          let_origem|meta_qtde_onibus|
+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+
|         22|    2532|            1|      8028-10|      METRÔ MORUMBI|        PARAISÓPOLIS|               5|
|         22|    2237|            1|      4056-10|   TERM. SÃO MATEUS|   PQ. BOA ESPERANÇA|               2|
|         22|   33121|            2|      5154-10|TERM. PRINC. ISABEL|    TERM. STO. AMARO|               5|
|         22|     683|            1|      271F-10|   METRÔ BELÉM     |        CENTER NORTE|               2|
|         22|      28|            1|      6818-10|    TERM. CAPELINHA|JD. VALE DAS VIRT...|               3|
+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+
only showing top 5 

In [346]:
df_meta_hora= df_meta_linha.where(col("hora_id_ref") == str_lasthour )

In [347]:
df_meta_hora.show(5)

+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+
|hora_id_ref|id_linha|sentido_linha|let_cod_linha|        let_destino|          let_origem|meta_qtde_onibus|
+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+
|         22|    2532|            1|      8028-10|      METRÔ MORUMBI|        PARAISÓPOLIS|               5|
|         22|    2237|            1|      4056-10|   TERM. SÃO MATEUS|   PQ. BOA ESPERANÇA|               2|
|         22|   33121|            2|      5154-10|TERM. PRINC. ISABEL|    TERM. STO. AMARO|               5|
|         22|     683|            1|      271F-10|   METRÔ BELÉM     |        CENTER NORTE|               2|
|         22|      28|            1|      6818-10|    TERM. CAPELINHA|JD. VALE DAS VIRT...|               3|
+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+
only showing top 5 

In [348]:
#####################################################################################################
#
# Realiza o left outer join entre as informações da meta e a média de ônibus em circulação
#

In [349]:
df_valida_qtde = df_meta_hora \
    .join(df_avg_linha_sel, ( df_meta_hora['hora_id_ref'] == df_avg_linha_sel['avg_hora_id_ref'] ) \
                              & ( df_meta_hora['id_linha'] == df_avg_linha_sel['avg_id_linha'] ) , 'left' ) \
    .drop( 'avg_hora_id_ref', 'avg_id_linha') \
    .fillna( {'avg_qtde_onibus':0,'data_ref': str_date} ) \
    .select( col("data_ref"), col("hora_id_ref"), col("id_linha"), col("sentido_linha"), col("let_cod_linha"), col("let_destino"), col("let_origem"), \
             col("meta_qtde_onibus"), col("avg_qtde_onibus") )
    

In [350]:
df_valida_qtde.show(5)

+--------+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+---------------+
|data_ref|hora_id_ref|id_linha|sentido_linha|let_cod_linha|        let_destino|          let_origem|meta_qtde_onibus|avg_qtde_onibus|
+--------+-----------+--------+-------------+-------------+-------------------+--------------------+----------------+---------------+
|20240920|         22|    2532|            1|      8028-10|      METRÔ MORUMBI|        PARAISÓPOLIS|               5|              3|
|20240920|         22|    2237|            1|      4056-10|   TERM. SÃO MATEUS|   PQ. BOA ESPERANÇA|               2|              2|
|20240920|         22|   33121|            2|      5154-10|TERM. PRINC. ISABEL|    TERM. STO. AMARO|               5|              4|
|20240920|         22|     683|            1|      271F-10|   METRÔ BELÉM     |        CENTER NORTE|               2|              2|
|20240920|         22|      28|            1|      6818-10|   

In [351]:
df_valida_qtde.where(col("avg_qtde_onibus") == 0).count()

15

In [352]:
#####################################################################################################
#
# Seleciona as linhas de ônibus que possuem um valor maior ou igual a meta estabelecida
#

In [353]:
df_lista_linhas_ok= df_valida_qtde.where( col("avg_qtde_onibus") >= col("meta_qtde_onibus") )

In [354]:
df_lista_linhas_ok.show(5)

+--------+-----------+--------+-------------+-------------+----------------+--------------------+----------------+---------------+
|data_ref|hora_id_ref|id_linha|sentido_linha|let_cod_linha|     let_destino|          let_origem|meta_qtde_onibus|avg_qtde_onibus|
+--------+-----------+--------+-------------+-------------+----------------+--------------------+----------------+---------------+
|20240920|         22|    2237|            1|      4056-10|TERM. SÃO MATEUS|   PQ. BOA ESPERANÇA|               2|              2|
|20240920|         22|     683|            1|      271F-10|METRÔ BELÉM     |        CENTER NORTE|               2|              2|
|20240920|         22|      28|            1|      6818-10| TERM. CAPELINHA|JD. VALE DAS VIRT...|               3|              3|
|20240920|         22|     490|            1|      715M-10| LGO. DA PÓLVORA|     JD. MARIA LUIZA|               6|              7|
|20240920|         22|     275|            1|      675P-10| METRÔ CONCEIÇÃO|     SH

In [355]:
#####################################################################################################
#
# Calcula a quantidade de linhas de ônibus que atingiram a meta
#

In [356]:
df_total_linhas_ok= df_lista_linhas_ok.groupby('data_ref','hora_id_ref').agg(count('sentido_linha').alias('qtde_linhas'))

In [357]:
df_total_linhas_ok.show()

+--------+-----------+-----------+
|data_ref|hora_id_ref|qtde_linhas|
+--------+-----------+-----------+
|20240920|         22|       1454|
+--------+-----------+-----------+



In [358]:
#####################################################################################################
#
# Seleciona as linhas de ônibus que ficaram abaixo da meta
#

In [359]:
df_lista_linhas_abaixo= df_valida_qtde.where( col("avg_qtde_onibus") < col("meta_qtde_onibus") )

In [360]:
df_lista_linhas_abaixo.show(5)

+--------+-----------+--------+-------------+-------------+-------------------+----------------+----------------+---------------+
|data_ref|hora_id_ref|id_linha|sentido_linha|let_cod_linha|        let_destino|      let_origem|meta_qtde_onibus|avg_qtde_onibus|
+--------+-----------+--------+-------------+-------------+-------------------+----------------+----------------+---------------+
|20240920|         22|    2532|            1|      8028-10|      METRÔ MORUMBI|    PARAISÓPOLIS|               5|              3|
|20240920|         22|   33121|            2|      5154-10|TERM. PRINC. ISABEL|TERM. STO. AMARO|               5|              4|
|20240920|         22|   34959|            2|      4018-10|   TERM. SÃO MATEUS|    METALÚRGICOS|              12|             11|
|20240920|         22|   34847|            2|      8072-10|      METRÔ MORUMBI|         PQ. IPÊ|               6|              5|
|20240920|         22|    1949|            1|      809L-10|               LAPA|     CAMPO 

In [361]:
#####################################################################################################
#
# Calcula a quantidade de linhas de ônibus que ficaram abaixo da meta do contrato
# de ônibus do contrato
#

In [362]:
df_total_linhas_abaixo= df_lista_linhas_abaixo.groupby('data_ref','hora_id_ref').agg(count('sentido_linha').alias('qtde_linhas'))

In [363]:
df_total_linhas_abaixo.show()

+--------+-----------+-----------+
|data_ref|hora_id_ref|qtde_linhas|
+--------+-----------+-----------+
|20240920|         22|        592|
+--------+-----------+-----------+



In [364]:
#####################################################################################################
#
# Gravação dos dataframe na camada Ouro em formato parquet:
# 1. Lista de linhas que atingiram a meta estabelecida
# 2. Lista de linhas que estão abaixo da meta estabelecida
# 3. Total de linhas que atingiram a meta estabelecida
# 4. Total de linhas que estão abaixo da meta estabelecida

In [365]:
df_lista_linhas_ok.write.parquet(ouro_lista_linhas_ok, mode='overwrite')

In [366]:
df_total_linhas_ok.write.parquet(ouro_total_linhas_ok, mode='overwrite')

In [367]:
df_lista_linhas_abaixo.write.parquet(ouro_lista_linhas_abaixo, mode='overwrite')

In [368]:
df_total_linhas_abaixo.write.parquet(ouro_total_linhas_abaixo, mode='overwrite')

In [369]:
#####################################################################################################
#
# Gravação dos dataframe na base de dados Postgres:
# 1. Lista de linhas que atingiram a meta estabelecida
# 2. Lista de linhas que estão abaixo da meta estabelecida
# 3. Total de linhas que atingiram a meta estabelecida
# 4. Total de linhas que estão abaixo da meta estabelecida
#

In [370]:
#gravação dos dataframes nas tabelas stages 

In [371]:
df_lista_linhas_ok.write.jdbc( url=url, table='sptrans.linhas_atingida_meta_stage',mode="overwrite", properties= properties)

In [372]:
df_total_linhas_ok.write.jdbc( url=url, table='sptrans.total_linhas_atingida_meta_stage',mode="overwrite", properties= properties)

In [373]:
df_lista_linhas_abaixo.write.jdbc( url=url, table='sptrans.linhas_abaixo_meta_stage',mode="overwrite", properties= properties)

In [374]:
df_total_linhas_abaixo.write.jdbc( url=url, table='sptrans.total_linhas_abaixo_meta_stage',mode="overwrite", properties= properties)

In [375]:
##################################################################
#
# upsert dos dados das stages nas tabelas fatos
#

In [376]:
import psycopg2
from psycopg2 import sql

In [377]:
#conexão na base de dados Postgres
conn = psycopg2.connect(
    dbname="dvdrental",
    user="admin",
    password="admin",
    host="db",
    port="5432"
)

# Criar um cursor
cur = conn.cursor()

In [378]:
# Upsert na tabela linhas_atingida_meta com as linhas da respectiva tabela stage

In [379]:
query = """
INSERT INTO sptrans.linhas_atingida_meta(data_ref, hora_id_ref, id_linha, sentido_linha, let_cod_linha, let_destino, let_origem, meta_qtde_onibus, avg_qtde_onibus)
   SELECT data_ref, hora_id_ref, id_linha, sentido_linha, let_cod_linha, let_destino, let_origem, meta_qtde_onibus, avg_qtde_onibus
   FROM sptrans.linhas_atingida_meta_stage
ON CONFLICT (data_ref, hora_id_ref, id_linha) 
DO UPDATE SET 
    sentido_linha = EXCLUDED.sentido_linha,
    let_cod_linha = EXCLUDED.let_cod_linha,
    let_destino = EXCLUDED.let_destino,
    let_origem = EXCLUDED.let_origem,
    meta_qtde_onibus = EXCLUDED.meta_qtde_onibus,
    avg_qtde_onibus = EXCLUDED.avg_qtde_onibus;
"""

In [380]:
# Executar a consulta
cur.execute(query)

In [381]:
# Commit das mudanças
#conn.commit()

In [382]:
# Upsert na tabela total_linhas_atingida_meta com as linhas da respectiva tabela stage

In [383]:
query = """
INSERT INTO sptrans.total_linhas_atingida_meta(data_ref, hora_id_ref, qtde_linhas)
   SELECT data_ref, hora_id_ref, qtde_linhas
   FROM sptrans.total_linhas_atingida_meta_stage
ON CONFLICT (data_ref, hora_id_ref) 
DO UPDATE SET 
    qtde_linhas = EXCLUDED.qtde_linhas;
"""

In [384]:
# Executar a consulta
cur.execute(query)

In [385]:
# Commit das mudanças
#conn.commit()

In [386]:
# Upsert na tabela linhas_abaixo_meta com as linhas da respectiva tabela stage

In [387]:
query = """
INSERT INTO sptrans.linhas_abaixo_meta(data_ref, hora_id_ref, id_linha, sentido_linha, let_cod_linha, let_destino, let_origem, meta_qtde_onibus, avg_qtde_onibus)
   SELECT data_ref, hora_id_ref, id_linha, sentido_linha, let_cod_linha, let_destino, let_origem, meta_qtde_onibus, avg_qtde_onibus
   FROM sptrans.linhas_abaixo_meta_stage
ON CONFLICT (data_ref, hora_id_ref, id_linha) 
DO UPDATE SET 
    sentido_linha = EXCLUDED.sentido_linha,
    let_cod_linha = EXCLUDED.let_cod_linha,
    let_destino = EXCLUDED.let_destino,
    let_origem = EXCLUDED.let_origem,
    meta_qtde_onibus = EXCLUDED.meta_qtde_onibus,
    avg_qtde_onibus = EXCLUDED.avg_qtde_onibus;
"""

In [388]:
# Executar a consulta
cur.execute(query)

In [389]:
# Commit das mudanças
#conn.commit()

In [390]:
# Upsert na tabela total_linhas_abaixo_meta com as linhas da respectiva tabela stage

In [391]:
query = """
INSERT INTO sptrans.total_linhas_abaixo_meta(data_ref, hora_id_ref, qtde_linhas)
   SELECT data_ref, hora_id_ref, qtde_linhas
   FROM sptrans.total_linhas_abaixo_meta_stage
ON CONFLICT (data_ref, hora_id_ref) 
DO UPDATE SET 
    qtde_linhas = EXCLUDED.qtde_linhas;
"""

In [392]:
# Executar a consulta
cur.execute(query)

In [393]:
# Commit das mudanças
conn.commit()

In [394]:
# Fechar o cursor e a conexão
cur.close()
conn.close()

In [395]:
#####################################################################################################
#
# Fim do processamento